In [1]:
import pandas as pd
import json
import re
import matplotlib.pyplot as plt
import numpy as np
import statistics
from collections import Counter
import plotly
import plotly.graph_objects as go
import plotly.express as px
import colorlover as cl
from itertools import cycle
import os


In [2]:
#Loading Data
results_filepath = "db_06_07_21_resp.json"
with open(results_filepath) as f:
#remove image0/image1 vars since it prevents proper df merging
    g = re.sub(r'_image[0-2]',"", f.read())
    data = json.loads(g)
    df = pd.json_normalize(data)

#hacky solution to remove the panda-appended user_. entry
renamed= [name.split("_.")[-1] for name in df.columns]
col_rename = {i:j for i,j in zip(df.columns,renamed)}
df = df.rename(columns=col_rename)

In [3]:
def createSubDFs(df):
    user_df = df[['user_profiling_age', 'user_profiling_position',
        'user_profiling_useOfDP', 'user_profiling_useOfAI',
        'user_profiling_useOfAI_details', 'user_profiling_mlFamiliarity']]

    one_df = df[['counterfactuals_prototypeInterpolation_understandability',
        'counterfactuals_prototypeInterpolation_usability',
        'counterfactuals_prototypeInterpolation_informativeness',
        'counterfactuals_prototypeInterpolation_value']]
    two_df = df[[ 'counterfactuals_twoAxisCounterfactuals_understandability',
        'counterfactuals_twoAxisCounterfactuals_usability',
        'counterfactuals_twoAxisCounterfactuals_informativeness',
        'counterfactuals_twoAxisCounterfactuals_value']]
    cf_names = ["counterfactuals_understandability","counterfactuals_usability","counterfactuals_informativeness","counterfactuals_value"]
    one_df.columns=cf_names
    two_df.columns=cf_names
    cf_df = pd.concat([one_df,two_df])

    ca_df = df[['conceptAttribution_textAttributes_understandability',
        'conceptAttribution_textAttributes_usability',
        'conceptAttribution_textAttributes_informativeness',
        'conceptAttribution_textAttributes_value',
        'conceptAttribution_textAttributes_comments']]

    pr_df = df[['prototypes_prototypes_understandability',
        'prototypes_prototypes_usability',
        'prototypes_prototypes_informativeness',
        'prototypes_prototypes_value']]

    ts_df = df[['trustScores_borderlineCases_understandability',
        'trustScores_borderlineCases_usability',
        'trustScores_borderlineCases_value',
        'trustScores_borderlineCases_informativeness',
        'trustScores_borderlineCases_comments']]

    #split the sm dfs so we unify different global/local values
    local_df = df[['saliencyMaps_localSaliency_understandability',
       'saliencyMaps_localSaliency_usability',
       'saliencyMaps_localSaliency_informativeness',
       'saliencyMaps_localSaliency_value',]]

    global_df = df[[  'saliencyMaps_globalSaliency_understandability',
       'saliencyMaps_globalSaliency_usability',
       'saliencyMaps_globalSaliency_informativeness',
       'saliencyMaps_globalSaliency_value']]
    sm_names = ["saliencyMaps_understandability","saliencyMaps_usability","saliencyMaps_informativeness","saliencyMaps_value"]
    local_df.columns=sm_names
    global_df.columns=sm_names
    sm_df = pd.concat([local_df,global_df])

    dfs = {
    "Saliency Map" : sm_df,
    "Concept Attribution" : ca_df,
    "Prototypes" : pr_df,
    "Counterfactuals" : cf_df,
    "Trust Scores" : ts_df
    }

    return user_df,dfs

In [4]:
user_df,dfs = createSubDFs(df)

In [5]:

def stackedBarChartDF(sub_df: pd.DataFrame, title: str,labels: list, palette, save_fig: bool = False, save_dir: str = "images"):
    #throw out non-numeric data
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    int_df = sub_df.select_dtypes(include=numerics).dropna()
    data = int_df.values

    #get the occurences in the respective column
    understandability = Counter(data[:,0])
    usability = Counter(data[:,1])
    value = Counter(data[:,2])
    informativeness = Counter(data[:,3])

    fig = go.Figure() #type: ignore
    category_order = ["Strongly disagree","Disagree","Slightly disagree","Neutral","Slightly agree","Agree","Strongly agree"]
    
    def add_bar_trace(num, x_format):
        num_order = [1,2,3,4,5,6,7]
        rating = num_order[num]
        counts = [understandability[rating],value[rating],informativeness[rating],usability[rating]]
        total_count = sum(understandability.values())

        xvals = [x_format(count)/total_count for count in counts]

        fig.add_trace(
            go.Bar(
                x=xvals,
                y=labels,
                orientation='h',
                name=category_order[num],
                # customdata=xvals,
                # hovertemplate = "%{y}: %{customdata}",
                width=0.8,
                marker_color=palette[num]
                    )) #type: ignore

    #negative side
    add_bar_trace(3, lambda x: x * -0.5)
    for num in reversed(range(0,3)):
        add_bar_trace(num, lambda x: x *-1)

    #positive side
    add_bar_trace(3, lambda x: x * 0.5)
    for num in range(4,7):
        add_bar_trace(num, lambda x: x)

    fig.update_layout(barmode='relative', 
                    yaxis_autorange='reversed',
                    legend_y=0.5,
                    title = title,
                    title_x = 0.45,
                    xaxis = {
                        "tick0" : 0,
                        "title": "Frequency",
                        "tickformat" : '%',
                        "range" : [-1,1]
                        },
                    showlegend=False
    )
    fig.show()

    if save_fig:
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        
        fig.write_image(f"{save_dir}/{name}.png")


#boxPlotDF(trust_df,"Trust Based Methods")

labels = ["Intuitiveness ","Relevance ","Trustworthiness ","Value "]
palette = [plotly.colors.diverging.RdBu[n] for n in [2,3,4,5,6,7,8]] #type: ignore

for name, df in dfs.items():
    stackedBarChartDF(df,name,labels, palette, save_fig=True, save_dir=os.path.splitext(results_filepath)[0])

#function to get most relevant statistical measures for single column
def descriptiveStatistics(column):
    print(df[column].describe())

#descriptiveStatistics("user_profiling_useOfDP")

In [9]:
#compare all approaches 

#1. step: create average,var score of each rating question per subdf
df_list = [value for value in dfs]

def applyFunctionToSubDF(df_list,func):
    avg_container = []
    #for each df, get an average of the ratings for understand,usab,value and info (or any other function you want to apply to the seletced subdf)
    for sub_df in df_list:
        understand_avg = round(func(sub_df.filter(regex=(".*_understandability")).values),2)
        usability_avg = round(func(sub_df.filter(regex=(".*_usability")).values),2)
        value_avg = round(func(sub_df.filter(regex=(".*_value")).values),2)
        info_avg = round(func(sub_df.filter(regex=(".*_informativeness")).values),2)
    
        sub_df_avg = [info_avg,understand_avg,usability_avg,value_avg]
        avg_container.append(sub_df_avg)

    #make sure that the input ordering in the df_list is the same as in the method call since I only rely on ordering, not on names
    return avg_container[0],avg_container[1],avg_container[2],avg_container[3],avg_container[4]

cf_avg,ca_avg,prototypes_avg,trust_avg,saliency_avg = applyFunctionToSubDF(df_list,np.mean)
cf_var,ca_var,prototypes_var,trust_var,saliency_var = applyFunctionToSubDF(df_list,np.var)

#2. step: compare them in a table with avg+sdv
comparison_df = pd.DataFrame(index=["Understandability","Usability","Value","Informativeness"])
comparison_df["TrustScores"] = trust_avg
comparison_df["CounterFactuals"] = cf_avg
comparison_df["ConceptAttribution"] = ca_avg
comparison_df["Prototypes"] = prototypes_avg
comparison_df["SaliencyMaps"] = saliency_avg
print(comparison_df)

#3. step: visualize them with boxplots
#for all df
def boxPlotDF(comparison_df,title,labels):
    #throw out non-numeric data
    int_df = comparison_df.select_dtypes(include="float64")
    #set the color palette
    palette = cycle(plotly.colors.sequential.Viridis)#type: ignore

    #iterate over columns and show average spread
    fig = go.Figure()#type: ignore
    for column in int_df:
        fig.add_trace(go.Box(
            y=int_df[column],
            name=column,
            marker_color=next(palette)
            )) #type: ignore
    
    fig.update_layout(
                    title = title,
                    yaxis = dict(
                        dtick = 1
                    ),
                    yaxis_range=[1,7],
                    yaxis_title="Average Rating"
    )
    
    fig.show()
    

boxPlotDF(comparison_df,"Comparison of Annotation Average",labels)


                   TrustScores  CounterFactuals  ConceptAttribution  \
Understandability         4.77             4.36                4.36   
Usability                 4.68             4.70                4.95   
Value                     4.23             4.57                4.82   
Informativeness           4.59             4.30                4.59   

                   Prototypes  SaliencyMaps  
Understandability        4.50          4.42  
Usability                5.73          4.45  
Value                    4.50          4.55  
Informativeness          4.59          4.05  
